In [2]:
import random
import cassiopeia as cass
from cassiopeia import Queue, Patch
from cassiopeia import Summoner, MatchHistory, Match
from cassiopeia.data import Continent
import arrow

### 1: Start

In [3]:
cass.set_riot_api_key("RGAPI-3fe686f2-d295-41d2-b0fb-97f13a95f0f4")
region = "EUW"

### 2: Pobranie 300 najlepszych graczy z challengera

In [4]:
best_300_challenger = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives, region=region)
best = best_300_challenger[0]
print(best.summoner)

Making call: https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5
Summoner(id=qioW32v1UCyc7X6m8ooydwef7HQegTRyZn3-IBAqNrJEDVg, account_id=?, name='Don Noway', puuid='?')


### 3: Pobranie 300 summonerów

In [5]:
summoners = []
for best in best_300_challenger:
    summoner = Summoner(name=best.summoner.name, region=region)
    summoners.append(summoner)

In [6]:
print(summoners[0].id, summoners[0].name)

qioW32v1UCyc7X6m8ooydwef7HQegTRyZn3-IBAqNrJEDVg Don Noway


### 4: Pobranie id championów

Unfortunately since we are iterating over the match history and accessing the summoner's champion for each match,
we need to know what version of the static data the champion should have. To avoid pulling many different
static data versions, we will instead create a {champion_id -> champion_name} mapping and just access the champion's
ID from the match data (which it provides directly).

In [7]:
champion_id_to_name_mapping = {
    champion.id: champion.name for champion in cass.get_champions(region=region)
}

Making call: https://ddragon.leagueoflegends.com/realms/euw.json
Making call: https://ddragon.leagueoflegends.com/cdn/12.23.1/data/en_GB/championFull.json


### 5: Pobieranie historii meczy

Zbierane są dane od patcha ??11.1?? do najnowszego.

In [8]:
patch = Patch.from_str("3.1", region=region)
print(patch.start)

#
def filter_match_history(summoner, patch):
    end_time = patch.end
    if end_time is None:
        end_time = arrow.now()
    match_history = MatchHistory(
        puuid=summoner.puuid,
        queue=Queue.ranked_solo_fives,
        start_time=patch.start,
        end_time=end_time,
        continent=Continent.europe
    )
    print('len match_history:', len(match_history))
    return match_history



Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
2013-07-30T03:00:00-02:00


In [ ]:
def get_matches_history(summoners, patch):
    matches_history = []
    for summoner in summoners:
        match_history = summoner.match_history
        print("Match history")
        for x in match_history:
            print(x)
        matches_history += match_history
        # print(match_history.count)
    return matches_history


matches_history = get_matches_history(summoners, patch)

Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/qioW32v1UCyc7X6m8ooydwef7HQegTRyZn3-IBAqNrJEDVg
Match history
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/rrr96D0D4vUy4z6bysNFNVtJgcP68iRkI9nnSscx4SFj4ropQcF-8C128mcQkxWyIpTpA7SV-wJAAQ/ids?start=0&count=100
Match(id=6185598768, region='EUROPE')
Match(id=6185541654, region='EUROPE')
Match(id=6185455862, region='EUROPE')
Match(id=6184695118, region='EUROPE')
Match(id=6184674930, region='EUROPE')
Match(id=6183918390, region='EUROPE')
Match(id=6183880017, region='EUROPE')
Match(id=6183828282, region='EUROPE')
Match(id=6183220660, region='EUROPE')
Match(id=6183180112, region='EUROPE')
Match(id=6183140279, region='EUROPE')
Match(id=6173687097, region='EUROPE')
Match(id=6173639136, region='EUROPE')
Match(id=6173585604, region='EUROPE')
Match(id=6173530803, region='EUROPE')
Match(id=6173481850, region='EUROPE')
Match(id=6169222882, region='EUROPE')
Match(id=6169170921, region='EUROPE')
Match(i

In [9]:
print('Matches length:', len(matches_history))
match_history = matches_history[1]
print(match_history.to_json())
# print(match_history.participants[0].summoner)
# print(len(matches_history))
# match_history

NameError: name 'matches_history' is not defined